In [2]:
api_key = "RGAPI-1d9435f9-62f2-45f2-9e6b-07991ce1c250"

In [3]:
import requests

version_patch = requests.get('https://ddragon.leagueoflegends.com/api/versions.json').json()
version_patch = version_patch[0]
data_champion = requests.get('https://ddragon.leagueoflegends.com/cdn/'+ version_patch +'/data/fr_FR/champion.json').json()
id_champion = data_champion["data"]

def getChampionInfo(id):
    global id_champion
    for key in id_champion:
        if id_champion[key]['key'] == str(id):
            return key
    return False

getChampionInfo(555)

'Pyke'

In [4]:
#converter roman number to int (the date won't go further 5 as there is only 5 tier possible in the data)
class roman_to_int(object):
   def romanToInt(self, s):
      """
      :type s: str
      :rtype: int
      """
      roman = {'I':1,'V':5}
      i = 0
      num = 0
      while i < len(s):
         if i+1<len(s) and s[i:i+2] in roman:
            num+=roman[s[i:i+2]]
            i+=2
         else:
            num+=roman[s[i]]
            i+=1
      return num


In [5]:
## function ranking_information : take a list of summoners name and return a list of those summoners but with their rank informations (if the summoners is not found he is not added to the output list)

def ranking_information (summoner_list_name:list):
    global list_summoner_rank
    list_summoner_rank = []

    for summoner in summoner_list_name:
    
        ## retrieve two important informations (the id and puuid) for the next steps
        
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        if data_summoner is not None and "id" in data_summoner:
            id_summoner = data_summoner.get("id")
            puuid_summoner = data_summoner.get("puuid")
          
            ## with the id we can retrieve the information of the account (tier, LP, rank)  
            
            data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
        
            ## we check from the data_rank if the account has done any SOLO Q games for this season only !!
            
            presence_of_soloq_data = False
            for data_SOLOQ in data_rank:
                if data_SOLOQ['queueType'] == "RANKED_SOLO_5x5":
                    print(summoner,data_SOLOQ['tier'],data_SOLOQ['rank'],data_SOLOQ['leaguePoints'], "LP")
                    rank = roman_to_int().romanToInt(data_SOLOQ['rank'])
                    
                    ## here we save the informations for each account            
                      
                    list_summoner_rank.append((summoner,data_SOLOQ['tier'],rank,data_SOLOQ['leaguePoints']))
                    presence_of_soloq_data = True
            if not presence_of_soloq_data:
                print(f"No data / No SOLO Q for this account : {summoner}")

                ## we still keep the summoner in the final list (to avoid users thinking that the account was not checked)
                    
                list_summoner_rank.append((summoner,"none","0", "0"))
        else:
            print(f"Failed to retrieve data for summoner {summoner}")
            
summoner_list = ["SaintLoutr","Draeknof","Haukeas Grémory","Caio73","charlos92604","7689000","Alrsay","TheK1ngPotato","Blarckful"]
#summoner_list = ["SaintLoutr","TheK1ngPotato","Draeknof"]

ranking_information(summoner_list)

SaintLoutr SILVER III 0 LP
Draeknof GOLD III 0 LP
Haukeas Grémory GOLD IV 0 LP
Caio73 BRONZE I 67 LP
Failed to retrieve data for summoner charlos92604
7689000 BRONZE IV 0 LP
Alrsay GOLD I 100 LP
TheK1ngPotato GOLD III 11 LP
Blarckful SILVER II 98 LP


In [6]:
import datetime
import calendar

## set the time of today

now = datetime.datetime.now()

start_of_day = datetime.datetime(
    year=now.year,
    month=now.month,
    day=now.day,
    hour=0,
    minute=0,
    second= 1,
)

end_of_day = datetime.datetime(
    year=now.year,
    month=now.month,
    day=now.day,
    hour=23,
    minute=59,
    second=59,
)

## retrieve the start of the day and the end of the day

end_of_day_epoch = str(int(end_of_day.timestamp()))
start_of_day_epoch = str(int(start_of_day.timestamp()))

print(start_of_day_epoch, end_of_day_epoch)


1679526001 1679612399


In [10]:
## the function result_match take a list of summoners and return a list of the results of their SOLOQ of the day 
def result_match (summoner_list):

    for summoner in summoner_list:
        w = 0
        l = 0 
        wr = 0
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        if "id" in data_summoner:
            puuid_summoner = data_summoner.get("puuid")
            
            ## with the id, the end of the day, and the start of the day we can check if the account has done any SOLOQ for the day
            
            matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&startTime='+ start_of_day_epoch +'&endTime='+ end_of_day_epoch +'&count=20&api_key='+api_key).json()

            ## check if there is any match
            
            if len(matches_id) != 0:  
                for match_id in matches_id:
                    
                    ## check the result of the game for the summoner
                    
                    match_info = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+ match_id +"?api_key=" + api_key).json()
                    for player in (match_info['info']['participants']):
                        if player['puuid'] == puuid_summoner:
                            if player['win'] == True:
                                w += 1
                            else:
                                l += 1
                wr = (w / (w + l)) * 100
                print(f"le winrate de {summoner}:",wr,"%","pour",w + l,"parties")
            else:
                print(f"Pas de match classé aujourd'hui pour {summoner}")            
    list_player = []

result_match(summoner_list)

Pas de match classé aujourd'hui pour SaintLoutr
Pas de match classé aujourd'hui pour Draeknof
le winrate de Haukeas Grémory: 0.0 % pour 1 parties
Pas de match classé aujourd'hui pour Caio73
Pas de match classé aujourd'hui pour 7689000
Pas de match classé aujourd'hui pour Alrsay
le winrate de TheK1ngPotato: 50.0 % pour 2 parties
Pas de match classé aujourd'hui pour Blarckful


In [ ]:
#ajout des données de rank des compte dans une base de données

import sqlite3

def insert_into_leaderboard(SummonerName, Tier, Rank, LP):
  try:
    conn = sqlite3.connect('leaderboard.db')
    cur = conn.cursor()
    print("Connexion réussie à SQLite")
    sql = "INSERT INTO leaderboard (SummonerName, Tier, Rank, LP) VALUES (?, ?, ?, ?)"
    value = (SummonerName, Tier, Rank, LP)
    cur.execute(sql, value)
    conn.commit()
    print(f"Enregistrement inséré avec succès dans la table leaderboard pour le compte {SummonerName}")
    cur.close()
    conn.close()
    print("Connexion SQLite est fermée")
  except sqlite3.Error as error:
    print("Erreur lors de l'insertion dans la table leaderboard", error)

for summoner in list_summoner_rank:
    SummonerName, Tier, Rank, LP = summoner
    insert_into_leaderboard(SummonerName, Tier, Rank, LP)

Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard pour le compte SaintLoutr
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard pour le compte TheK1ngPotato
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard pour le compte Draeknof
Connexion SQLite est fermée


In [ ]:
#suppresion de la table

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("delete from leaderboard")
   conn.commit()
   print("leaderboard supprimé")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)

Connexion réussie à SQLite
leaderboard supprimé
Connexion SQLite est fermée


In [11]:
#sorting de la liste

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("
SELECT *FROM data_ranking ORDER BY CASE 
WHEN Rank = 'CHALLENGER' THEN 1 
WHEN Rank = 'GRANDMASTER' THEN 2 
WHEN Rank = 'MASTER' THEN 3 
WHEN Rank = 'DIAMOND' THEN 4 
WHEN Rank = 'PLATINIUM' THEN 5 
WHEN Rank = 'GOLD' THEN 6 
WHEN Rank = 'SILVER' THEN 7 
WHEN Rank = 'BRONZE' THEN 8 
WHEN Rank = 'IRON' THEN 9 
ELSE 10 END
,Tier asc,lp desc;
")
   conn.commit()
   print("leaderboard sorted")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)

SyntaxError: unterminated string literal (detected at line 7) (2098873530.py, line 7)